In [28]:
import json
from objectbox import (
    Entity,
    Id,
    String,
    Store,
    Box,
    Float32Vector,
    HnswIndex,
    VectorDistanceType,
    
)
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pathlib import Path
import yaml

In [29]:
rules_chunks_pth = Path("../data/podzemelja_chunks.cleaned.json")
rules_chunks: list[dict] = json.loads(rules_chunks_pth.read_text())
term_ner_pth = Path("../data/terms_merged_ru.json")
term_ner: list[dict] = json.loads(term_ner_pth.read_text())

In [30]:
rules_chunks[0]

{'id': 'subtitle',
 'content': 'ПОДЗЕМЕЛЬЯ ПЁСИКИ\n\nПравила игры',
 'type': 'subtitle',
 'section': 'main_title',
 'merged_from': 'title',
 'isHeader': False}

In [31]:
term_ner[0]

{'id': 'c3d5b7b9-0dbd-4277-b540-13966774a9c6',
 'name': 'Исследование',
 'slug': 'explore',
 'kind': 'TERM',
 'path': 'game_mechanics/actions/explore',
 'group': 'game_mechanics',
 'definition': 'Взять верхнюю карту комнат и выложить её на стол',
 'extra': {'restrictions': ['хотя бы один выход должен совпадать',
   'карту можно вращать']}}

In [32]:
for rule in rules_chunks:
    tags = " ".join(
        [
            "#game:Подземелье_и_Песики",
            f"#section:{rule['section']}",
            f"#type:{rule['type']}",
        ]
    )
    rule["content"] = f"{tags}\n---\n{rule["content"]}"

In [33]:
print(rules_chunks[2]["content"])

#game:Подземелье_и_Песики #section:introduction #type:narrative
---
В волшебном королевстве каждый год проходят соревнования по побегу из подземелья. Самые хорошие и смелые пёсики спускаются в глубокие мрачные комнаты, чтобы сразиться с чудовищами и собрать как можно больше трофеев. Победителей ждут почести и слава, титулы и горы золота, а также очень вкусные косточки.


In [34]:
for term in term_ner:
    tags = " ".join(["#game:Подземелье_и_Песики", f"#group:{term["group"]}"])
    term["content"] = f"{tags}\n---\n{term["name"]}"
    term["extra"] = yaml.safe_dump(term["extra"], allow_unicode=True)

In [35]:
term_ner[0]

{'id': 'c3d5b7b9-0dbd-4277-b540-13966774a9c6',
 'name': 'Исследование',
 'slug': 'explore',
 'kind': 'TERM',
 'path': 'game_mechanics/actions/explore',
 'group': 'game_mechanics',
 'definition': 'Взять верхнюю карту комнат и выложить её на стол',
 'extra': 'restrictions:\n- хотя бы один выход должен совпадать\n- карту можно вращать\n',
 'content': '#game:Подземелье_и_Песики #group:game_mechanics\n---\nИсследование'}

In [36]:
@Entity()
class Rule:
    id = Id
    internal_id = String
    content = String
    section = String
    game = String
    vector = Float32Vector(
        index=HnswIndex(dimensions=768, distance_type=VectorDistanceType.EUCLIDEAN)
    )
    
@Entity()
class Terminology:
    id = Id
    internal_id = String
    content = String
    name = String
    slug = String
    kind = String
    path = String
    group = String
    definition = String
    extra = String
    vector = Float32Vector(
        index = HnswIndex(dimensions=768)
    )

In [37]:
store = Store(directory="../db")
rules_box = Box(store=store, entity=Rule)
term_ner_box = Box(store, Terminology)

---

In [38]:
model = SentenceTransformer("../model").to("cpu")

In [39]:
box_objects = [
    Rule(
        internal_id=obj["id"],
        content=obj["content"],
        section=obj["section"],
        game="Подземелье и пёсики",
        vector=model.encode(obj["content"]).tolist(),
    )
    for obj in tqdm(rules_chunks)
]

100%|██████████| 51/51 [00:10<00:00,  4.86it/s]


In [ ]:
box_terms = [
    Terminology(
        internal_id=obj["id"],
        content=obj["content"],
        name=obj["name"],
        slug=obj["slug"],
        kind=obj["kind"],
        path=obj["path"],
        group=obj["group"],
        definition=obj["definition"],
        extra=obj["extra"],
        vector=model.encode(obj["content"]).tolist(),
    )
    for obj in tqdm(term_ner)
]

In [41]:
rules_box.put(box_objects)
term_ner_box.put(box_terms)

In [42]:
store.close()